In [1]:
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import json
import os
from os import listdir, getcwd
from os.path import isfile, join
import re
from ultralytics import YOLO
import pybboxes as pbx
import torch

In [2]:
class Coordinates:
    x1 = 0
    y1 = 0
    x2 = 0
    y2 = 0
    imgsize = 0

    def __init__(self, imgsize, coor):
        self.x1 = coor[0][0]
        self.y1 = coor[0][1]
        self.x2 = coor[1][0]
        self.y2= coor[1][1]
        self.imgsize = imgsize
    
    def rotate90degrees(self):
        new_x1 = self.y1
        new_y1 = self.imgsize - self.x2
        new_x2 = self.y2
        new_y2 = self.imgsize - self.x1

        return Coordinates(self.imgsize, [[new_x1, new_y1], [new_x2, new_y2]]) 

    def convert_to_yolo(self):
        return pbx.convert_bbox((self.x1, self.y1, self.x2, self.y2), from_type='voc', to_type='yolo', image_size=(self.imgsize, self.imgsize))

def extract_coordinates_from_json(j):
    cattle_coordinates = [a['exterior'] for a in [p['points'] for p in j['objects']]]
    return [Coordinates(3000, c) for c in cattle_coordinates]

def show_image(image, cattle_coordinates):
    img = np.asarray(image)
    plt.figure()
    plt.imshow(img)
    ax = plt.gca()
    for c in cattle_coordinates:
        width = c.x2 - c.x1
        height = c.y2 - c.y1

        rect = Rectangle((c.x1,c.y1),width,height,linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
    plt.show()

def write_coordinates_in_yolo_file(filepath, coordinates):
    with open(filepath, 'w') as out_file:
        for c in coordinates:
            yolo_format = c.convert_to_yolo()
            out_file.write(str(0) + " " + " ".join([str(a) for a in yolo_format]) + '\n')

## Show example image

In [ ]:
image_file = Image.open('dataset/Dataset1/img/auto2_DJI_0062.JPG')
cattle_coordinates = []
with open('dataset/Dataset1/ann/auto2_DJI_0062.JPG.json') as f:
    j = json.load(f)
    cattle_coordinates = extract_coordinates_from_json(j)

show_image(image_file, cattle_coordinates)

## Check if there are coordinates where the image will be croped

In [5]:
ann_path = "dataset/Dataset1/ann"

count = 0
for f in listdir(ann_path):
    cattle_coordinates = []
    with open(f'{ann_path}/{f}') as f:
        j = json.load(f)
        cattle_coordinates = [p['points'] for p in j['objects']]
    for c in cattle_coordinates:
        top_left_x = c['exterior'][0][0]
        bottom_right_x = c['exterior'][1][0]

        if (top_left_x < 1000 and bottom_right_x > 1000) or (top_left_x < 3000 and bottom_right_x > 3000):
            count+=1

print(count)    

68


## Generate dataset

Divide the image in two and create different variations by rotating the image

In [16]:
def generate_dataset(source_images_path, source_ann_path, target_image_path, target_ann_path, save=False, display_image = True):
    box1 = (0, 0, 3000, 3000)
    box2 = (1000, 0, 4000, 3000)

    for f in listdir(source_images_path):
        img = Image.open(f'{source_images_path}/{f}')
        cattle_coordinates = []
        with open(f'{source_ann_path}/{f}.json') as f_ann:
            j = json.load(f_ann)
            cattle_coordinates = extract_coordinates_from_json(j)

        try:
            first_half = img.crop(box1)
            second_half = img.crop(box2)
        except:
            print(f'Unable to crop image: {f}')
            continue
        file_name, file_extension = os.path.splitext(str(f))
        
        first_half_90 = first_half.rotate(90)
        first_half_180 = first_half.rotate(180)
        second_half_90 = second_half.rotate(90)
        second_half_180 = second_half.rotate(180)

        # save images
        if save:
            first_half.save(f'{target_image_path}/{file_name}_1{file_extension}')
            first_half_90.save(f'{target_image_path}/{file_name}_1_90r{file_extension}')
            first_half_180.save(f'{target_image_path}/{file_name}_1_180r{file_extension}')
            second_half.save(f'{target_image_path}/{file_name}_2{file_extension}')
            second_half_90.save(f'{target_image_path}/{file_name}_2_90r{file_extension}')
            second_half_180.save(f'{target_image_path}/{file_name}_2_180r{file_extension}')
            
        # transfer coordinates
        first_half_coordinates = [c for c in cattle_coordinates if c.x1 < 3000]
        
        # has ann after the 1000 threshold or has some leftovers  
        second_half_coordinates = [c for c in cattle_coordinates if c.x1 >= 1000 or (c.x1 < 1000 and c.x2 > 1000)]
        _break = False

        if first_half_coordinates:
            for c in first_half_coordinates:
                # check if the box ends after the threshold
                if c.x2 > 3000:
                    c.x2 = 3000 - 1
        
            coordinates_90r = [c.rotate90degrees() for c in first_half_coordinates]
            coordinates_180r = [c.rotate90degrees() for c in coordinates_90r]

            if display_image:
                show_image(first_half, first_half_coordinates)
                show_image(first_half_90, coordinates_90r)
                show_image(first_half_180, coordinates_180r)

            txtpath = f'{target_ann_path}/{file_name}_1.txt'
            txtpath2 = f'{target_ann_path}/{file_name}_1_90r.txt'
            txtpath3 = f'{target_ann_path}/{file_name}_1_180r.txt'
            if save:
                write_coordinates_in_yolo_file(txtpath, first_half_coordinates)
                write_coordinates_in_yolo_file(txtpath2, coordinates_90r)
                write_coordinates_in_yolo_file(txtpath3, coordinates_180r)

            
        if second_half_coordinates:
            for c in second_half_coordinates:
                c.x1 -= 1000
                c.x2 -= 1000

                # check if box was cropped
                if c.x1 < 0:
                    c.x1 = 0

            coordinates_90r = [c.rotate90degrees() for c in second_half_coordinates]
            coordinates_180r = [c.rotate90degrees() for c in coordinates_90r]
            
            if display_image:
                show_image(second_half, second_half_coordinates)
                show_image(second_half_90, coordinates_90r)
                show_image(second_half_180, coordinates_180r)

            txtpath = f'{target_ann_path}/{file_name}_1.txt'
            txtpath2 = f'{target_ann_path}/{file_name}_2_90r.txt'
            txtpath3 = f'{target_ann_path}/{file_name}_2_180r.txt'
        
            if save:
                write_coordinates_in_yolo_file(txtpath, second_half_coordinates)
                write_coordinates_in_yolo_file(txtpath2, coordinates_90r)
                write_coordinates_in_yolo_file(txtpath3, coordinates_180r)
            
                
        if _break:
            print('breakk')
            break

In [18]:
train_images_path = "dataset/Dataset1/img"
train_ann_path = "dataset/Dataset1/ann"

target_train_image_path = "dataset_fixed/images/train"
target_train_ann_path = "dataset_fixed/labels/train"

generate_dataset(train_images_path, train_ann_path, target_train_image_path, target_train_ann_path, save=True, display_image=False)


val_images_path = "dataset/Dataset2/img"
val_ann_path = "dataset/Dataset2/ann"

target_val_image_path = "dataset_fixed/images/val"
target_val_ann_path = "dataset_fixed/labels/val"

generate_dataset(val_images_path, val_ann_path, target_val_image_path, target_val_ann_path, save=True, display_image=False)

Unable to crop image: auto4_DJI_0002.JPG
Unable to crop image: auto6_DJI_0002.JPG
Unable to crop image: auto7_DJI_0002.JPG


## Train model

In [3]:
# Load YOLOv10n model from scratch

#print(torch.cuda.is_available())
model = YOLO("yolov8s.pt")



# results = model('dataset_fixed/images/val/DJI_0028_1_180r.JPG')
# results[0].show()
results = model.train(data="dataset.yaml", epochs=100, imgsz=768)

Ultralytics YOLOv8.2.87  Python-3.12.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=768, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning C:\Users\jvito\cattle_counting\dataset_fixed\labels\train.cache... 1348 images, 2570 backgrounds, 0 corrupt: 100%|██████████| 3918/3918 [00:00<?, ?it/s]
val: Scanning C:\Users\jvito\cattle_counting\dataset_fixed\labels\val.cache... 60 images, 24 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<?, ?it/s]


Plotting labels to runs\detect\train27\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 768 train, 768 val
Using 8 dataloader workers
Logging results to runs\detect\train27
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      5.77G      1.887      5.329      1.062         58        768: 100%|██████████| 245/245 [01:36<00:00,  2.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.47it/s]

                   all         84        260      0.472      0.362      0.326      0.102


...

    100/100      5.76G      1.019     0.7203     0.8688         37        768: 100%|██████████| 245/245 [01:16<00:00,  3.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.48it/s]

                   all         84        260      0.465      0.323      0.285      0.165



100 epochs completed in 2.422 hours.
Optimizer stripped from runs\detect\train27\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train27\weights\best.pt, 22.5MB

Validating runs\detect\train27\weights\best.pt...
Ultralytics YOLOv8.2.87  Python-3.12.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.53it/s]


                   all         84        260      0.604      0.596      0.571      0.302
Speed: 0.3ms preprocess, 6.5ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train27


## Run inference

In [14]:
#Image.open('runs/detect/train11/results.png')
#Image.open('runs/detect/train11/val_batch0_pred.jpg')
model = YOLO("lucas_runs/runs/detect/train14/weights/best.pt")
results = model('drone-view-of-running-horses-in-a-green-field-video.mp4', show=True)


WARNING  lucas_runs/runs/detect/train14/weights/best.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "dill" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "dill" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "dill" ' returned non-zero exit status 1.


ModuleNotFoundError: No module named 'dill'